##### b) 不可变数据类

In [ ]:
@dataclass(frozen=True)
class APICredentials:
    api_key: str
    api_secret: str
    
# 创建后无法修改
credentials = APICredentials("test_key", "test_secret")
try:
    credentials.api_key = "new_key"  # 将引发异常
except dataclasses.FrozenInstanceError:
    print("无法修改不可变数据类的属性")

##### c) 继承与组合

In [ ]:
@dataclass
class BaseTestCase:
    case_id: str
    name: str
    method: str
    
@dataclass
class KYCTestCase(BaseTestCase):
    api_url: str
    body: Dict[str, Any]
    headers: Optional[Dict[str, str]] = None
    params: Optional[Dict[str, str]] = None
    
# 使用继承的数据类
test_case = KYCTestCase(
    case_id="KYC_001",
    name="客户身份验证测试",
    method="POST",
    api_url="/verify",
    body={
        "customer_id": "KYC-12345",
        "document_type": "passport"
    }
)

#### 3. 数据验证与转换

##### a) 类型检查

In [ ]:
from dataclasses import dataclass
from typing import Union, List
import datetime

@dataclass
class CustomerDocument:
    doc_type: str
    doc_number: str
    issue_date: datetime.date
    expiry_date: datetime.date
    
    def __post_init__(self):
        """验证文档类型"""
        valid_types = ["passport", "id_card", "driver_license"]
        if self.doc_type not in valid_types:
            raise ValueError(f"无效的证件类型: {self.doc_type}")
        
        # 验证日期
        if self.expiry_date <= self.issue_date:
            raise ValueError("过期日期必须晚于签发日期")

# 使用示例
try:
    doc = CustomerDocument(
        doc_type="passport",
        doc_number="P12345678",
        issue_date=datetime.date(2020, 1, 1),
        expiry_date=datetime.date(2030, 1, 1)
    )
except ValueError as e:
    print(f"验证错误: {e}")

##### b) 数据转换

In [ ]:
from dataclasses import dataclass, asdict, astuple
from typing import Dict

@dataclass
class APIResponse:
    status_code: int
    message: str
    data: Dict[str, Any]
    
    def to_dict(self) -> Dict[str, Any]:
        """转换为字典"""
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'APIResponse':
        """从字典创建实例"""
        return cls(**data)

# 使用示例
response = APIResponse(
    status_code=200,
    message="验证成功",
    data={"customer_id": "KYC-12345", "status": "verified"}
)

# 转换为字典
response_dict = response.to_dict()
print(response_dict)

# 从字典创建实例
new_response = APIResponse.from_dict(response_dict)
print(new_response)

#### 4. 实际应用：配置管理

让我们看看如何在KYC测试框架中使用数据类来管理配置：

In [ ]:
from dataclasses import dataclass
from typing import Dict, Optional, List
import yaml
from pathlib import Path

@dataclass
class TestEnvironment:
    name: str
    base_url: str
    timeout: int = 30
    verify_ssl: bool = True
    headers: Optional[Dict[str, str]] = None

@dataclass
class TestCase:
    case_id: str
    name: str
    method: str
    api_url: str
    body: Optional[Dict[str, Any]] = None
    headers: Optional[Dict[str, str]] = None
    params: Optional[Dict[str, str]] = None
    expected_status: int = 200

@dataclass
class TestConfig:
    environments: Dict[str, TestEnvironment]
    test_cases: Dict[str, TestCase]
    default_environment: str
    
    @classmethod
    def from_yaml(cls, yaml_path: Path) -> 'TestConfig':
        """从YAML文件加载配置"""
        with open(yaml_path, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
            
        # 转换环境配置
        environments = {
            name: TestEnvironment(**env_data)
            for name, env_data in data['environments'].items()
        }
        
        # 转换测试用例
        test_cases = {
            case_id: TestCase(case_id=case_id, **case_data)
            for case_id, case_data in data['test_cases'].items()
        }
        
        return cls(
            environments=environments,
            test_cases=test_cases,
            default_environment=data['default_environment']
        )

# 使用示例
config = TestConfig.from_yaml(Path('test_config.yaml'))
print(f"默认环境: {config.default_environment}")
print(f"测试用例数量: {len(config.test_cases)}")

#### 5. 最佳实践

1. **类型提示**：始终使用类型提示，提高代码可读性和IDE支持
2. **默认值**：为可选字段提供合理的默认值
3. **验证逻辑**：在 `__post_init__` 中实现数据验证
4. **不可变性**：对于配置类，考虑使用 `frozen=True` 确保不可变
5. **文档字符串**：为数据类和字段添加清晰的文档说明
6. **转换方法**：实现与其他数据格式（如字典、YAML）的转换方法
7. **继承使用**：合理使用继承来复用共同属性
8. **组合优于继承**：在适当的情况下使用组合而不是继承

#### 6. 练习题

1. 实现一个 `RequestBuilder` 数据类，用于构建HTTP请求
2. 为 `TestCase` 数据类添加结果验证功能
3. 实现配置的版本控制功能
4. 添加敏感信息（如API密钥）的加密处理
5. 实现测试用例的导入导出功能

#### 7. 小结

在本节中，我们学习了Python数据类的基本概念和使用方法。数据类为我们提供了一种简洁、类型安全的方式来处理配置数据，特别适合用于测试框架的配置管理。通过使用数据类，我们可以：

1. 减少样板代码，提高代码可读性
2. 利用类型提示进行静态类型检查
3. 实现数据验证和转换
4. 确保配置数据的一致性和可靠性

在下一节中，我们将学习如何将YAML配置和数据类结合使用，实现一个完整的配置管理模块。